<a href="https://colab.research.google.com/github/rabierp/GCP-ML-APIs-Labs/blob/main/Vision_NLP_Translate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialisation Projet

In [ ]:
!gcloud auth login pascalr@google.com

In [ ]:
!gcloud config get-value core/account

In [ ]:
!gcloud config set project pascal-demo

In [ ]:
!gcloud services list --enabled --filter=vision

In [ ]:
!gcloud services list --enabled --filter=language

In [ ]:
!gcloud services list --enabled --filter=translate

In [ ]:
!gcloud iam service-accounts create my-lab-sa --display-name "my lab service account"

In [ ]:
PROJECT = !gcloud config get-value core/project
%env GOOGLE_CLOUD_PROJECT {PROJECT[0]}

In [ ]:
!gcloud iam service-accounts keys create key.json --iam-account my-lab-sa@${GOOGLE_CLOUD_PROJECT}.iam.gserviceaccount.com

In [ ]:
%%bash
pwd
ls

In [ ]:
%env GOOGLE_APPLICATION_CREDENTIALS "key.json"

# Détection de "Point de repère" par appel de l'API Vision avec _curl_

La détection de point de repère ("_LANDMARK DETECTION_") dans une image est une des fonctionnalités de l'API Cloud Vision.

Dans cette section, vous allez analyser [cette image](https://storage.googleapis.com/cloud-vision-codelab/eiffel_tower.jpg), préalablement chargée pour vous dans un _bucket_ **Google Cloud Storage** (vous pouvez cliquer sur le lien pour visualiser l'image).

## Préparation du corps de la requête API

Vous allez d'abord construire la requete HTTP pour appeler l'API Vision, puis envoyer cette requête avec la commande en ligne curl (commande shell Linux / Unix).

Ecrivez le corps de votre requete en JSON dans le fichier request.json :

In [ ]:
%%bash
cat <<EOF > request.json
{
  "requests": [
    {
      "image": {
        "source": {
              "gcsImageUri": "gs://cloud-vision-codelab/eiffel-tower.jpg"
          }
      },
      "features": [
        {
          "maxResults": 10,
          "type": "LANDMARK_DETECTION"
        },
      ]
    }
  ]
}
EOF

Vérifiez le contenu du fichier 'request.json' :

In [ ]:
!cat request.json

## Exécution de l'appel API avec la commande _curl_

La commande _curl_ est une commande en ligne (CLI) disponible depuis le shell d'un système Linux / Uni, voire Windows si les outils adéquats ont été préalablement installés.

In [ ]:
!pwd
!curl -X POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @request.json \
https://vision.googleapis.com/v1/images:annotate

# Appel de l'API Vision avec le _client API_ Python

In [ ]:
!pip install google-cloud-vision

In [ ]:
!python3 -c "import google.cloud.vision"

# Appel de l'API Natural Language

# Appel API Translation

In [ ]:
!pip install google-cloud-translate